In [30]:
import pandas as pd
import os
import torch
import networkx as nx
from torch_geometric.utils import to_networkx

In [3]:
problems = ["bongard","krk","train","samegen","university","mutag","nci","cancer","financial","ptc"]
base_path = os.path.join("docker","Benchmark")

for problem in problems:
    df = pd.read_csv(os.path.join(base_path,problem,"relational",f"{problem}.csv"))
    print(f"{problem} has {len(df)} examples")

bongard has 392 examples
krk has 200 examples
train has 400 examples
samegen has 200 examples
university has 27 examples
mutag has 188 examples
nci has 200 examples
cancer has 330 examples
financial has 234 examples
ptc has 343 examples


In [4]:
# create the dataset features latex table: features as row and the datasets with representations as columns
datasets = ["mutag","nci","cancer","financial","ptc"]
#datasets = ["mutag"]
#datasets = ["krk","bongard","train","sameGen","cyclic"]

# average amount of nodes per example
representations = ["node_only","node_edge","edge_based","Klog"]

# average amount of nodes
print("Avg. nodes",end=" & ")
for dataset in datasets:
    all_nodes = {}
    for representation in representations:
        # if exists, load the graph data
        if os.path.exists(os.path.join(base_path,dataset,representation,"graph","train.pt")):
            graphs = torch.load(os.path.join(base_path,dataset,representation,"graph","train.pt"))
            nodes = sum([len(graph.x) for graph in graphs])/len(graphs)
            all_nodes[representation] = nodes
        else:
            all_nodes[representation] = 0
    
    values = list(all_nodes.values())
    for value in values:
        print(f"{round(value)}",end=" & ")
print("\\\\")

# average amount of edges
print("Avg. edges",end=" & ")
for dataset in datasets:
    all_edges = {}
    for representation in representations:
        # if exists, load the graph data
        if os.path.exists(os.path.join(base_path,dataset,representation,"graph","train.pt")):
            graphs = torch.load(os.path.join(base_path,dataset,representation,"graph","train.pt"))
            edges = sum([len(graph.edge_index[0]) for graph in graphs])/len(graphs)
            all_edges[representation] = edges
        else:
            all_edges[representation] = 0
    
    values = list(all_edges.values())
    for value in values:
        print(f"{round(value)}",end=" & ")
print("\\\\")

# amount of examples, same value for all representations so use multicolumn 
print("Size",end=" & ")
for dataset in datasets:
    graphs = torch.load(os.path.join(base_path,dataset,"node_only","graph","train.pt"))
    size = len(graphs)
    print("\multicolumn{4}{c|}{"+str(size)+"}",end=" & ")
print("\\\\")

# Graph diameter -> do this with networkx
print("Depth",end=" & ")
for dataset in datasets:
    all_depths = {}
    for representation in representations:
        # if exists, load the graph data
        if os.path.exists(os.path.join(base_path,dataset,representation,"graph","train.pt")):
            graphs = torch.load(os.path.join(base_path,dataset,representation,"graph","train.pt"))
            depths = []
            for graph in graphs:
                G = to_networkx(graph).to_undirected()
                try:
                    #depth = nx.diameter(G)
                    depths.append(nx.diameter(G))
                except:
                    #S = [G.subgraph(c).copy() for c in nx.connected_components(G)]
                    S = G.subgraph(max(nx.connected_components(G), key=len)).copy()
                    #depths = [nx.diameter(s) for s in S]
                    depths.append(nx.diameter(S))
                #depths.append(max(depths))
            all_depths[representation] = sum(depths)/len(depths)
        else:
            all_depths[representation] = 0
    
    values = list(all_depths.values())
    for value in values:
        print(f"{round(value)}",end=" & ")
print("\\\\")
    
# Graph width -> # node features
print("Width",end=" & ")
for dataset in datasets:
    all_widths = {}
    for representation in representations:
        # if exists, load the graph data
        if os.path.exists(os.path.join(base_path,dataset,representation,"graph","train.pt")):
            graphs = torch.load(os.path.join(base_path,dataset,representation,"graph","train.pt"))
            widths = sum([len(graph.x[0]) for graph in graphs])/len(graphs)
            all_widths[representation] = widths
        else:
            all_widths[representation] = 0
    
    values = list(all_widths.values())
    for value in values:
        print(f"{round(value)}",end=" & ")
print("\\\\")
         

Avg. nodes & 59 & 26 & 27 & 58 & 114 & 37 & 40 & 117 & 83 & 27 & 34 & 83 & 404 & 395 & 0 & 400 & 54 & 27 & 31 & 81 & \\
Avg. edges & 3635 & 56 & 108 & 151 & 16217 & 160 & 234 & 320 & 22471 & 111 & 166 & 223 & 354293 & 8 & 0 & 15 & 8019 & 108 & 162 & 217 & \\
Size & \multicolumn{4}{c|}{148} & \multicolumn{4}{c|}{160} & \multicolumn{4}{c|}{260} & \multicolumn{4}{c|}{184} & \multicolumn{4}{c|}{270} & \\
Depth & 1 & 9 & 2 & 12 & 1 & 13 & 5 & 27 & 1 & 9 & 7 & 18 & 

KeyboardInterrupt: 

In [ ]:
# average amount of edges per example
representations = ["node_only","node_edge","edge_based","Klog"]
representations = ["node_only","node_edge","Klog"]
for problem in problems:
    print(f"Problem: {problem}")
    all_edges = {}
    for representation in representations:
        graphs = torch.load(os.path.join(base_path,problem,representation,"graph","train.pt"))
        edges = sum([len(graph.edge_index[0])/2 for graph in graphs])/len(graphs)
        all_edges[representation] = edges
    print(all_edges)
    values = list(all_edges.values())
    print(f"Max edges: {max(values)}")
    print(f"Min edges: {min(values)}")  

Problem: bongard
{'node_only': 15.70873786407767, 'node_edge': 1.7864077669902914, 'Klog': 3.5728155339805827}
Max edges: 15.70873786407767
Min edges: 1.7864077669902914
Problem: krk
{'node_only': 3.0, 'node_edge': 3.0, 'Klog': 7.7594936708860756}
Max edges: 7.7594936708860756
Min edges: 3.0
Problem: train
{'node_only': 36.52215189873418, 'node_edge': 7.674050632911392, 'Klog': 15.348101265822784}
Max edges: 36.52215189873418
Min edges: 7.674050632911392
Problem: samegen
{'node_only': 1.0, 'node_edge': 39.56962025316456, 'Klog': 78.39240506329114}
Max edges: 78.39240506329114
Min edges: 1.0
Problem: university
{'node_only': 41.333333333333336, 'node_edge': 2.761904761904762, 'Klog': 2.761904761904762}
Max edges: 41.333333333333336
Min edges: 2.761904761904762
Problem: mutag
{'node_only': 1817.587837837838, 'node_edge': 28.006756756756758, 'Klog': 75.74324324324324}
Max edges: 1817.587837837838
Min edges: 28.006756756756758
Problem: nci
{'node_only': 8108.3625, 'node_edge': 79.974683544

In [32]:
def get_logic_results(problem):
    base_path = os.path.join("docker","Benchmark")
    df = pd.read_csv(os.path.join(base_path,problem,"results","results_logic_final.csv"))
    return df

def get_gnn_results(problem):
    base_path = os.path.join("docker","Benchmark")
    df = pd.read_csv(os.path.join(base_path,problem,"results","results_gnn_final.csv"))
    return df

def get_kernel_results(problem):
    base_path = os.path.join("docker","Benchmark")
    df = pd.read_csv(os.path.join(base_path,problem,"results","kernel_results_final.csv"))
    return df

In [31]:
# proces over the representations

dataset_name = "mutag"
#df_gnn = get_gnn_results(dataset_name)
df_gnn = pd.read_csv(os.path.join("docker","Benchmark",dataset_name,"results","results_gnn_final.csv"))

representations = ["node_only","node_edge","edge_based","Klog"] 
#representations = ["node_edge","edge_based","Klog"]

# group the gnn test_acc by representation
grouped_test_acc = df_gnn.groupby(by=["representation","model"])["test_acc"].mean()
grouped_std = df_gnn.groupby(by=["representation","model"])["test_acc_std"].mean()

print("GNN")
for repr in representations:
    # best model for this representation
    best_model = grouped_test_acc[repr].idxmax()
    print("repr",repr)
    print(f"{round(100*grouped_test_acc[repr][best_model],2)}\pm{round(100*grouped_std[repr][best_model],2)}")

#df_logic = get_logic_results(dataset_name)
df_logic = pd.read_csv(os.path.join("docker","Benchmark",dataset_name,"results","results_logic_final_1.csv"))
print("\n")
print("Logic")
grouped_test_acc = df_logic.groupby(by=["representation"])["test_acc"].max()
for repr in representations:
    print("repr",repr)
    print(f"{round(100*grouped_test_acc[repr],2)}")
            


GNN
repr node_only
68.67\pm0.62
repr node_edge
69.31\pm1.25
repr edge_based
67.87\pm0.96
repr Klog
68.61\pm1.18


FileNotFoundError: [Errno 2] No such file or directory: 'docker/Benchmark/mutag/results/results_logic_final_1.csv'

In [ ]:
# process over the models/systems per representation
dataset_name = "krk"
#df_logic = get_logic_results(dataset_name)
#df_logic = pd.read_csv(os.path.join("docker","Benchmark",dataset_name,"results","results_logic.csv"))
#df_gnn = get_gnn_results(dataset_name)
df_gnn = pd.read_csv(os.path.join("docker","Benchmark",dataset_name,"results","results_gnn_dist.csv"))
df_gnn = pd.read_csv(os.path.join("docker","Benchmark",dataset_name,"results","results_gnn_final.csv"))

representations = ["node_only","node_edge","edge_based","Klog"] #+ ["FullBoard","FullDiag"] #+ ["VirtualNode"]
#representations = ["node_edge"]
# group the gnn test_acc by representation
grouped_test_acc = df_gnn.groupby(by=["representation","model"])["test_acc"].max()
grouped_std = df_gnn.groupby(by=["representation","model"])["test_acc_std"].mean()

print("GNN: ")
for repr in representations:
    print("repr",repr)
    for model in df_gnn["model"].unique():
        print(f"{model}:    {round(100*grouped_test_acc[repr][model],2)}\pm{round(100*grouped_std[repr][model],2)}")

print("\n")
print("Logic: ")
grouped_test_acc = df_logic.groupby(by=["representation","system"])["test_acc"].max()
for repr in representations:
    print("repr",repr)
    for system in df_logic["system"].unique():
        print(f"{system}:    {round(100*grouped_test_acc[repr][system],2)}")
    

GNN: 
repr node_only
GINE:    76.19\pm2.91
GCN:    73.02\pm3.03
GlobalAttentionNet:    74.6\pm1.79
SAGPool:    74.6\pm1.81
repr node_edge
GINE:    82.54\pm1.86
GCN:    74.6\pm2.02
GlobalAttentionNet:    72.22\pm1.51
SAGPool:    73.02\pm2.06
repr edge_based
GINE:    91.27\pm2.03
GCN:    72.22\pm3.34
GlobalAttentionNet:    72.22\pm3.18
SAGPool:    72.22\pm3.45
repr Klog
GINE:    88.89\pm1.58
GCN:    89.68\pm1.05
GlobalAttentionNet:    88.89\pm0.84
SAGPool:    90.48\pm1.33


Logic: 


NameError: name 'df_logic' is not defined

In [42]:
# graph kernel results

dataset = "train"
dataset = "university"
dataset = "bongard"
dataset = "cyclic"
#datasets = ["krk","bongard","train","sameGen","cyclic"]
datasets = ["mutag","nci","cancer","financial","ptc"]

representations = ["node_only","node_edge","edge_based","Klog"]
#representations = ["node_edge","edge_based","Klog"]

# first go over the GNN representations
print("\midrule")
for repr in representations:
    print(f"{repr} & ",end="")
    for dataset in datasets:
        df_gnn = get_kernel_results(dataset)
        df_gnn_grouped = df_gnn.groupby(by=["representation","graph_kernel"])["test_accuracy"].mean()
        # get the best model
        if repr in df_gnn_grouped:
            best_model = df_gnn_grouped[repr].idxmax()
            #print("best_model",best_model)
            if df_gnn_grouped[repr][best_model] == 1.0:
                # take the second best model
                best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[1]
                if df_gnn_grouped[repr][best_model] == 1.0:
                    # take the third best model
                    best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[2]
            
            df_gnn_grouped_std = df_gnn.groupby(by=["representation","graph_kernel"])["test_accuracy"].std()
            # if last column, do not print the &
            print(f"${round(100*df_gnn_grouped[repr][best_model],2)}\pm{round(100*df_gnn_grouped_std[repr][best_model],2)}$ & ",end="")
        else:
            print(" & ",end="")
    print("\\\\")


# print the best model for th graph kernels
print("\midrule")
for repr in representations:
    print(f"{repr} & ",end="")
    for dataset in datasets:
        df_gnn = get_kernel_results(dataset)
        df_gnn_grouped = df_gnn.groupby(by=["representation","graph_kernel"])["test_accuracy"].mean()
        # get the best model
        if repr in df_gnn_grouped:
            best_model = df_gnn_grouped[repr].idxmax()
            if best_model == "weisfeiler_lehman":
                best_model = "WL"
            elif best_model == "graphlet_sampling":
                best_model = "GS"
            elif best_model == "shortest_path":
                best_model = "SP"
            # if last column, do not print the &
            print(f"{best_model} & ",end="")
    print("\\\\")


\midrule
node_only &  &  &  &  &  & \\
node_edge & $67.5\pm0.0$ & $76.19\pm0.0$ & $58.57\pm0.0$ & $91.49\pm0.0$ & $63.01\pm0.0$ & \\
edge_based & $67.5\pm0.0$ & $66.67\pm0.0$ & $60.48\pm2.97$ &  & $54.79\pm0.0$ & \\
Klog & $67.5\pm0.0$ & $78.57\pm0.0$ & $51.43\pm0.0$ & $91.49\pm0.0$ & $62.56\pm3.16$ & \\
\midrule
node_only & \\
node_edge & GS & WL & WL & WL & WL & \\
edge_based & SP & WL & GS & SP & \\
Klog & GS & WL & WL & WL & WL & \\


In [5]:
def print_latex_table(dataset):

    # GNN rows: the rows are the model and the columns the representations (node_only, node_edge, edge_based, Klog)
    df_gnn = get_gnn_results(dataset)
    #df_gnn = pd.read_csv(os.path.join("docker","Benchmark",dataset,"results","results_gnn.csv"))
    df_gnn_grouped = df_gnn.groupby(by=["representation","model"])["test_acc"].mean()
    df_gnn_grouped_std = df_gnn.groupby(by=["representation","model"])["test_acc_std"].mean()

    representations = ["node_only","node_edge","edge_based","Klog"]
    representations = ["node_edge","edge_based","Klog"]
    #representations = ["node_edge","Klog"]

    print("\midrule")
    for model in df_gnn["model"].unique():
        print(f"{model} & ",end="")
        for repr in representations:
            # if last column, do not print the &
            if repr == representations[-1]:
                print(f"${round(100*df_gnn_grouped[repr][model],2)}\pm{round(100*df_gnn_grouped_std[repr][model],2)}$ ",end="")
            else:
                print(f"${round(100*df_gnn_grouped[repr][model],2)}\pm{round(100*df_gnn_grouped_std[repr][model],2)}$ & ",end="")
        print("\\\\")

    # Logic rows: the rows are the system and the columns the representations
    df_logic = get_logic_results(dataset)

    print("\midrule")
    for system in df_logic["system"].unique():
        print(f"{system} & ",end="")
        df_logic_grouped = df_logic.groupby(by=["representation","system"])["test_acc"].max()
        for repr in representations:
            # if last column, do not print the &
            if repr == representations[-1]:
                print(f"${round(100*df_logic_grouped[repr][system],2)}$ ",end="")
            else:
                print(f"${round(100*df_logic_grouped[repr][system],2)}$ & ",end="")
        print("\\\\")
    
    

print_latex_table("cancer")


\midrule
GINE & $61.49\pm2.34$ & $57.58\pm1.32$ & $59.72\pm2.3$ \\
GCN & $62.5\pm2.08$ & $59.22\pm3.52$ & $59.6\pm1.32$ \\
GlobalAttentionNet & $61.87\pm1.76$ & $57.45\pm0.54$ & $60.98\pm1.33$ \\
SAGPool & $60.23\pm2.27$ & $59.6\pm2.31$ & $59.72\pm1.67$ \\


FileNotFoundError: [Errno 2] No such file or directory: 'docker/Benchmark/cancer/results/results_logic_final.csv'

In [6]:
# create the full latex table with the datasets as columns and the representations per method as rows

datasets = ["krk","bongard","train","sameGen","cyclic"]
datasets = ["mutag","nci","cancer","financial","ptc"]
representations = ["node_only","node_edge","edge_based","Klog"]


# first go over the GNN representations
print("\midrule")
for repr in representations:
    print(f"{repr} & ",end="")
    for dataset in datasets:
        df_gnn = get_gnn_results(dataset)
        df_gnn_grouped = df_gnn.groupby(by=["representation","model"])["test_acc"].mean()
        # get the best model
        if repr in df_gnn_grouped:
            best_model = df_gnn_grouped[repr].idxmax()
            if df_gnn_grouped[repr][best_model] == 1.0:
                # take the second best model
                best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[1]
                if df_gnn_grouped[repr][best_model] == 1.0:
                    # take the third best model
                    best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[2]
            
            df_gnn_grouped_std = df_gnn.groupby(by=["representation","model"])["test_acc_std"].mean()
            # if last column, do not print the &
            print(f"${round(100*df_gnn_grouped[repr][best_model],2)}\pm{round(100*df_gnn_grouped_std[repr][best_model],2)}$ & ",end="")
        else:
            print(" & ",end="")
    print("\\\\")

# then go over the logic representations
# print("\midrule")
# for repr in representations:
#     print(f"{repr} & ",end="")
#     for dataset in datasets:
#         df_logic = get_logic_results(dataset)
#         df_logic_grouped = df_logic.groupby(by=["representation"])["test_acc"].mean()
#         # if last column, do not print the &
#         if dataset == datasets[-1]:
#             print(f"${round(100*df_logic_grouped[repr],2)}$ ",end="")
#         else:
#             print(f"${round(100*df_logic_grouped[repr],2)}$ & ",end="")
#     print("\\\\")

# then go over the kernel representations
# print("\midrule")
# for repr in representations:
#     print(f"{repr} & ",end="")
#     for dataset in datasets:
#         df = pd.read_csv(os.path.join(base_path,dataset,"results","kernel_results.csv"))
#         grouped_test_acc = df.groupby(by=["representation",])["test_accuracy"].max()
#         grouped_std = df.groupby(by=["representation"])["test_accuracy"].std()
#         # if last column, do not print the &
#         if dataset == datasets[-1]:
#             print(f"${round(100*grouped_test_acc[repr],2)}\pm{round(100*grouped_std[repr],2)}$ ",end="")
#         else:
#             print(f"${round(100*grouped_test_acc[repr],2)}\pm{round(100*grouped_std[repr],2)}$ & ",end="")
#     print("\\\\")


# create a latex tabel that shows the best model or system per dataset and representation
datasets = ["krk","bongard","train","sameGen","cyclic"]
datasets = ["mutag","nci","cancer","financial","ptc"]
representations = ["node_only","node_edge","edge_based","Klog"]
print("\\\\")
print("\midrule")

for repr in representations:
    print(f"{repr} & ",end="")
    for dataset in datasets:
        df_gnn = get_gnn_results(dataset)
        df_gnn_grouped = df_gnn.groupby(by=["representation","model"])["test_acc"].mean()
        # get the best model
        if repr in df_gnn_grouped:
            best_model = df_gnn_grouped[repr].idxmax()
            if df_gnn_grouped[repr][best_model] == 1.0:
                # take the second best model
                best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[1]
                if df_gnn_grouped[repr][best_model] == 1.0:
                    # take the third best model
                    best_model = df_gnn_grouped[repr].sort_values(ascending=False).index[2]
        else:
            best_model = ""
        
        print(f"{best_model} & ",end="")
    print("\\\\")

# print("\midrule")
# for repr in representations:
#     print(f"{repr} & ",end="")
#     for dataset in datasets:
#         df_logic = get_logic_results(dataset)
#         df_logic_grouped = df_logic.groupby(by=["representation","system"])["test_acc"].max()
#         # if multiple systems have the same score take all of them
#         best_systems = df_logic_grouped[repr][df_logic_grouped[repr] == df_logic_grouped[repr].max()].index
#         # put the best systems in a string seperated by ,
#         best_system = ",".join(best_systems)
#         print(f"{best_system} & ",end="")
#     print("\\\\")




\midrule
node_only & $68.67\pm0.62$ & $76.46\pm3.52$ &  &  & $64.21\pm1.88$ & \\
node_edge & $69.31\pm1.25$ & $86.25\pm3.63$ & $62.5\pm2.08$ & $87.94\pm0.75$ & $65.46\pm1.68$ & \\
edge_based & $67.87\pm0.96$ & $88.75\pm2.79$ & $59.6\pm2.31$ &  & $64.49\pm3.74$ & \\
Klog & $68.61\pm1.18$ & $88.12\pm1.77$ & $60.98\pm1.33$ & $87.59\pm0.5$ & $65.22\pm2.25$ & \\
\\
\midrule
node_only & GINE & GINE &  &  & GINE & \\
node_edge & GINE & GINE & GCN & GINE & GINE & \\
edge_based & GINE & GINE & SAGPool &  & GINE & \\
Klog & GINE & GINE & GlobalAttentionNet & GINE & GINE & \\
